In [448]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import shutil
import glob
import os
from unicodedata import *
import time
import pandas as pd
import re
import plotly.express as px

In [449]:
driver = webdriver.Chrome()
driver.maximize_window()

# Directly to xlsx download link
driver.get('https://www.dshs.texas.gov/sites/default/files/chs/data/COVID/Texas%20COVID-19%20New%20Confirmed%20Cases%20by%20County.xlsx') 

In [450]:
# Generate a list of xlsx files in Downloads
list_of_files = glob.glob('/Users/zacshe/Downloads/*.xlsx')
# print(list_of_files)

# Return the lastest xlsx file in Downloads
latest_file = max(list_of_files, key= os.path.getmtime)
# print(latest_file)

# Generate a list of xlxs files in Python/Auto_data_downloads_TX directory
list_of_files = glob.glob('/Users/zacshe/Desktop/Python/Auto_data_downloads_TX/*.xlsx')

# Delete existing xlsx file
for x in list_of_files:
    print('deleting old xlsx file', x)
    os.remove(x)

# Mover the latest xlsx downloaded in Downloads to Python/Auto_data_downloads_TX
shutil.move(latest_file, '/Users/zacshe/Desktop/Python/Auto_data_downloads_TX')

# Generate a list of xlsx in the following directory
list_of_files = glob.glob('/Users/zacshe/Desktop/Python/Auto_data_downloads_TX/*.xlsx')

# Rename the file
os.rename(list_of_files[0].split('/')[6],'TX_COVID19_New_Cases.xlsx')

# File we are targeting
target_file = glob.glob('/Users/zacshe/Desktop/Python/Auto_data_downloads_TX/*.xlsx')[0].split('/')[6]

deleting old xlsx file /Users/zacshe/Desktop/Python/Auto_data_downloads_TX/TX_COVID19_New_Cases.xlsx


In [451]:
df = pd.read_excel(target_file, header=None)

# Drop rows and columns containing unwanted data
clean_df = df.drop(index=[0,1,257,258,259,260], columns=[302,303])

# Transpose the rows and columns
clean_df_t = clean_df.T

# Make the 1st row the header
clean_df_t.columns = clean_df_t.iloc[0]

# Drop the duplicate first row that has become the header 
clean_df_t = clean_df_t.iloc[1:]

# clean_df_t.tail(10)

In [452]:
clean_df_t.rename(columns={'County':'Date'}, inplace=True)

# Set ['Date'] column to datetime value
clean_df_t['Date'] = pd.to_datetime(clean_df_t['Date'])
# clean_df_t.head()

In [453]:
# The following code is same as: anderson = clean_df_t[['Date','Anderson']]
anderson = clean_df_t.iloc[:,[0,1]]

# Assign a new 'Case_Count' column with values from clean_df_t['Anderson']
anderson = anderson.assign(Case_Count = clean_df_t.iloc[:,1].values)

# Replace all values in ['Anderson'] column with 'Anderson'
anderson.iloc[:,1] = clean_df_t.columns[1]

# Rename column
anderson.rename(columns={anderson.columns[1]:'County'}, inplace=True)

# anderson.head()

In [454]:
df_list = []    # Empty list

for x in range(1,clean_df_t.shape[1]): # Range from the 1st column to the last
    a = clean_df_t.iloc[:,[0,x]]       # Retrieve first column ['Date'] and the Xth column (County)
    b = a.assign(Case_Count = clean_df_t.iloc[:,x].values)  # Create a new column of ['Case_Count'] with values from the ['County'] column
    b.iloc[:,1] = clean_df_t.columns[x]                     # Replace the values in ['County'] column with the name of county from the original Xth coloumn name
    b.rename(columns={b.columns[1]:'County'}, inplace=True) # Rename the county column with 'County'
    
    df_list.append(b)                                       # Append each dateframe into the empty list

concat_df = pd.concat(df_list, ignore_index=True)           # Concatenate the list vertically by column names

concat_df.to_csv('time_series_plotly.csv', index=False)     # Export as a CSV file

In [455]:
fig = px.scatter(concat_df, x='Date', y='Case_Count', color='County')
fig.update_traces(mode='markers+lines')
fig.show()